<a href="https://colab.research.google.com/github/SIDIBEMoussa/KaggmeCompete/blob/main/Efs_time_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import zipfile
zip_file_path = '/content/drive/MyDrive/Kaggle_Compete/Efs_prediction/equity-post-HCT-survival-predictions.zip'
unzip_directory = '/content/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_directory)

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

In [8]:
dict_data = pd.read_csv("data_dictionary.csv")

In [9]:
dict_data.head()

,variable,description,type,values
0,dri_score,Refined disease risk index,Categorical,['Intermediate' 'High' 'N/A - non-malignant in...
1,psych_disturb,Psychiatric disturbance,Categorical,['Yes' 'No' nan 'Not done']
2,cyto_score,Cytogenetic score,Categorical,['Intermediate' 'Favorable' 'Poor' 'TBD' nan '...
3,diabetes,Diabetes,Categorical,['No' 'Yes' nan 'Not done']
4,hla_match_c_high,Recipient / 1st donor allele level (high resol...,Numerical,NaN


In [10]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
sub_data = pd.read_csv("sample_submission.csv")

In [11]:
train_data.shape,test_data.shape,sub_data.shape

((28800, 60), (3, 58), (3, 2))

In [15]:
test_data

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10
0,28800,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,90.0,No,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0
1,28801,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0
2,28802,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0


In [16]:
train_data.nunique()

,0
ID,28800
dri_score,11
psych_disturb,3
cyto_score,7
diabetes,3
hla_match_c_high,3
hla_high_res_8,7
tbi_status,8
arrhythmia,3
hla_low_res_6,5


In [17]:
cat_col = train_data.columns[(train_data.nunique()<=18).values]

train_data[cat_col] = train_data[cat_col].astype("category")
cat_col = cat_col.difference(["efs"])
test_data[cat_col] = test_data[cat_col].astype("category")

In [18]:
100*train_data.isna().sum().sort_values(ascending=False)/train_data.shape[0]

,0
tce_match,65.958333
mrd_hct,57.628472
cyto_score_detail,41.399306
tce_div_match,39.569444
tce_imm_match,38.656250
cyto_score,28.013889
hla_high_res_10,24.871528
hla_high_res_8,20.239583
hla_high_res_6,18.347222
hla_match_dqb1_high,18.052083


In [19]:
def select_na_col(df,s1,s2):
    '''This function takes data and return columns according
    to nan percentage range'''
    if s2>0:
        na_percentage = 100*df.isna().sum()/df.shape[0]
        selection = ((na_percentage>s1) & (na_percentage<=s2)).values
        selected = df.columns[selection]

        return selected
    clean_columns = df.columns[(df.isna().sum()==0).values]
    return clean_columns

def separate_nan_rows(df):
    """Sépare les lignes avec NaN et non-NaN dans un DataFrame pandas.

    Args:
        df: Le DataFrame pandas.

    Returns:
        Un tuple contenant deux DataFrames : le premier avec les lignes contenant des NaN,
        et le second avec les lignes ne contenant pas de NaN.
    """
    nan_rows = df[df.isnull().any(axis=1)]
    not_nan_rows = df[df.notnull().all(axis=1)]
    return not_nan_rows,nan_rows

In [20]:
select_na_col(train_data,0,0)
complete_data = train_data[select_na_col(train_data,0,0)]
nan_data_to_treat_first = train_data[select_na_col(train_data,0,1)]
nan_data_to_treat_first.head(2)

,dri_score,vent_hist,in_vivo_tcd,gvhd_proph,sex_match,donor_related
0,N/A - non-malignant indication,No,Yes,FKalone,M-F,Unrelated
1,Intermediate,No,No,Other GVHD Prophylaxis,F-F,Related


In [21]:
complete_data.head(2)

,ID,tbi_status,graft_type,prim_disease_hct,prod_type,year_hct,age_at_hct,race_group,efs,efs_time
0,0,No TBI,Bone marrow,IEA,BM,2016,9.942,More than one race,0.0,42.356
1,1,"TBI +- Other, >cGy",Peripheral blood,AML,PB,2008,43.705,Asian,1.0,4.672


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,mean_absolute_percentage_error

In [23]:
def fillna(complete_data,nan_data_to_treat_first):

    complete_data_coded = pd.get_dummies(complete_data)
    cols = nan_data_to_treat_first.columns

    for col in cols:
        label_coding = LabelEncoder()
        complete_data_coded[col] = nan_data_to_treat_first[col]
        train,test = separate_nan_rows(complete_data_coded)
        if train[col].dtypes!='category':
            X,y = train.drop(columns=["ID",col]),train[col]
            X_topredict = test.drop(columns=["ID",col])
            X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=10)
            model = RandomForestRegressor( n_estimators=300,
                                           criterion='squared_error',
                                           max_depth=5,
                                          )
            model.fit(X_train,y_train)
            report = mean_absolute_percentage_error(y_test,model.predict(X_test))
        else:
            train[col]=label_coding.fit_transform(train[col])
            X,y = train.drop(columns=["ID",col]),train[col]
            X_topredict = test.drop(columns=["ID",col])
            X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=10)
            model = RandomForestClassifier(n_estimators=100,
                                           criterion='gini',
                                           max_depth=5,
                                           class_weight= "balanced"
                                          )
            model.fit(X_train,y_train)
            report = classification_report(y_test,model.predict(X_test))

        print(f"**************************{col}**************************")
        print(report)
        prediction = model.predict(X_topredict)
        test[col] = prediction
        complete_data_coded = pd.concat([train,test],axis=0)
    return complete_data_coded

In [24]:
temp = fillna(complete_data,train_data[select_na_col(train_data,0,42)])

**************************dri_score**************************
              precision    recall  f1-score   support

           0       0.50      0.67      0.57         3
           1       0.00      0.00      0.00         0
           2       0.67      0.40      0.50         5
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0

    accuracy                           0.40        10
   macro avg       0.19      0.18      0.18        10
weighted avg       0.48      0.40      0.42        10

**************************psych_disturb**************************
              precision    recall  f1-score   support

           0       1.00      0.60      0.75        10
           2       0.00      0.00      0.00         0

    accuracy                           0.60        10
   macro avg       0.50      0.30      0.38        10
weighted avg       1.00      0.60      0.75        10

**

In [25]:
train_data.shape,temp.shape

((28800, 60), (28800, 102))

In [ ]:
common_cols = complete_data.columns.difference(["efs","efs_time"])



In [ ]:
sub_data.head()